# Commission Models and Custom Commissions

This notebook demonstrates:

1. **Percentage Commission** - Most common (e.g., 0.1% per trade)
2. **Flat Commission** - Fixed fee per trade (e.g., $5 per trade)
3. **Tiered Commission** - Different rates for different trade sizes
4. **Custom Commission** - Creating your own commission structure
5. **Impact Analysis** - How commissions affect strategy performance

Understanding commission costs is **critical** for realistic backtesting!

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

from simple_backtest import BacktestConfig, Backtest
from simple_backtest.strategy import MovingAverageStrategy
from simple_backtest.commission import Commission, PercentageCommission, FlatCommission, TieredCommission
from simple_backtest.visualization import plot_equity_curve

## Load Data

In [2]:
# Download data
ticker = "SPY"
data = yf.download(ticker, start="2020-01-01", end="2023-12-31", progress=False)
data = data.dropna()

print(f"Data shape: {data.shape}")
print(f"Date range: {data.index[0]} to {data.index[-1]}")

/var/folders/9y/pp82h2912h3781myzdncvs_r0000gp/T/ipykernel_55185/3318573726.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2020-01-01", end="2023-12-31", progress=False)


Data shape: (1006, 5)
Date range: 2020-01-02 00:00:00 to 2023-12-29 00:00:00


# Download data
ticker = "SPY"
data = yf.download(ticker, start="2020-01-01", end="2023-12-31", progress=False)

# Handle MultiIndex columns if present
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)

data = data.dropna()

print(f"Data shape: {data.shape}")
print(f"Date range: {data.index[0]} to {data.index[-1]}")

In [3]:
# Test percentage commission
config_percentage = BacktestConfig(
    initial_capital=10000.0,
    lookback_period=50,
    commission_type="percentage",
    commission_value=0.001,  # 0.1% per trade
    risk_free_rate=0.02
)

print("Percentage Commission Configuration:")
print(f"  Commission: {config_percentage.commission_value * 100}% per trade")
print(f"\nExamples:")
print(f"  Trade $1,000:  Commission = ${1000 * 0.001:.2f}")
print(f"  Trade $5,000:  Commission = ${5000 * 0.001:.2f}")
print(f"  Trade $10,000: Commission = ${10000 * 0.001:.2f}")

Percentage Commission Configuration:
  Commission: 0.1% per trade

Examples:
  Trade $1,000:  Commission = $1.00
  Trade $5,000:  Commission = $5.00
  Trade $10,000: Commission = $10.00


In [4]:
# Run backtest with percentage commission
strategy = MovingAverageStrategy(short_window=10, long_window=30, shares=10)

backtest = Backtest(data=data, config=config_percentage)
results_percentage = backtest.run([strategy])

metrics = results_percentage[strategy.get_name()]['metrics']
print(f"\nPerformance with {config_percentage.commission_value*100}% Commission:")
print(f"  Total Return:  {metrics['total_return']*100:.2f}%")
print(f"  Total Trades:  {metrics['total_trades']}")
print(f"  Sharpe Ratio:  {metrics['sharpe_ratio']:.2f}")

Running strategies: 100%|██████████| 1/1 [00:00<00:00, 13.64it/s]


Performance with 0.1% Commission:
  Total Return:  1089.29%
  Total Trades:  31
  Sharpe Ratio:  0.19


## 2. Flat Commission

**Fixed fee per trade**, regardless of size.

Examples:
- Robinhood: $0 (free trading)
- Traditional brokers: $5-$10 per trade
- Some forex brokers: $0-$5

In [5]:
# Test flat commission
config_flat = BacktestConfig(
    initial_capital=10000.0,
    lookback_period=50,
    commission_type="flat",
    commission_value=5.0,  # $5 per trade
    risk_free_rate=0.02
)

print("Flat Commission Configuration:")
print(f"  Commission: ${config_flat.commission_value:.2f} per trade")
print(f"\nExamples:")
print(f"  Trade $1,000:  Commission = ${config_flat.commission_value:.2f}")
print(f"  Trade $5,000:  Commission = ${config_flat.commission_value:.2f}")
print(f"  Trade $10,000: Commission = ${config_flat.commission_value:.2f}")

Flat Commission Configuration:
  Commission: $5.00 per trade

Examples:
  Trade $1,000:  Commission = $5.00
  Trade $5,000:  Commission = $5.00
  Trade $10,000: Commission = $5.00


In [6]:
# Run backtest with flat commission
backtest = Backtest(data=data, config=config_flat)
results_flat = backtest.run([strategy])

metrics = results_flat[strategy.get_name()]['metrics']
print(f"\nPerformance with ${config_flat.commission_value:.2f} Flat Commission:")
print(f"  Total Return:  {metrics['total_return']*100:.2f}%")
print(f"  Total Trades:  {metrics['total_trades']}")
print(f"  Sharpe Ratio:  {metrics['sharpe_ratio']:.2f}")

Running strategies: 100%|██████████| 1/1 [00:00<00:00, 14.35it/s]


Performance with $5.00 Flat Commission:
  Total Return:  1053.69%
  Total Trades:  31
  Sharpe Ratio:  0.17


## 3. Tiered Commission

**Volume-based pricing**: Larger trades get lower commission rates.

Common for:
- Institutional brokers
- High-volume traders
- Some crypto exchanges

In [7]:
# Test tiered commission
# Define tiers: (threshold, rate)
tiers = [
    (1000, 0.002),          # $0-$1,000: 0.2%
    (5000, 0.001),          # $1,000-$5,000: 0.1%
    (float('inf'), 0.0005), # $5,000+: 0.05%
]

config_tiered = BacktestConfig(
    initial_capital=10000.0,
    lookback_period=50,
    commission_type="tiered",
    commission_value=tiers,
    risk_free_rate=0.02
)

print("Tiered Commission Configuration:")
print("  Tiers:")
print("    $0 - $1,000:    0.2% commission")
print("    $1,000 - $5,000: 0.1% commission")
print("    $5,000+:         0.05% commission")

# Calculate examples manually
print(f"\nExamples:")
print(f"  Trade $500:    ${500 * 0.002:.2f} (all in tier 1)")
print(f"  Trade $3,000:  ${1000 * 0.002 + 2000 * 0.001:.2f} (tier 1 + tier 2)")
print(f"  Trade $8,000:  ${1000 * 0.002 + 4000 * 0.001 + 3000 * 0.0005:.2f} (all tiers)")

Tiered Commission Configuration:
  Tiers:
    $0 - $1,000:    0.2% commission
    $1,000 - $5,000: 0.1% commission
    $5,000+:         0.05% commission

Examples:
  Trade $500:    $1.00 (all in tier 1)
  Trade $3,000:  $4.00 (tier 1 + tier 2)
  Trade $8,000:  $7.50 (all tiers)


In [8]:
# Run backtest with tiered commission
backtest = Backtest(data=data, config=config_tiered)
results_tiered = backtest.run([strategy])

metrics = results_tiered[strategy.get_name()]['metrics']
print(f"\nPerformance with Tiered Commission:")
print(f"  Total Return:  {metrics['total_return']*100:.2f}%")
print(f"  Total Trades:  {metrics['total_trades']}")
print(f"  Sharpe Ratio:  {metrics['sharpe_ratio']:.2f}")

Running strategies: 100%|██████████| 1/1 [00:00<00:00, 14.63it/s]


Performance with Tiered Commission:
  Total Return:  1058.29%
  Total Trades:  31
  Sharpe Ratio:  0.17


## 4. Custom Commission Models

Create your own commission structure by inheriting from the `Commission` base class.

Let's create two custom models:
1. **Minimum Fee Commission**: Percentage with a minimum fee
2. **Time-Based Commission**: Different rates for day vs overnight positions

In [9]:
class MinimumFeeCommission(Commission):
    """Percentage commission with a minimum fee.
    
    Common structure: Pay percentage or minimum fee, whichever is higher.
    Example: 0.1% or $5 minimum.
    """
    
    def __init__(self, rate: float, min_fee: float, name: str = None):
        super().__init__(name=name or f"MinFee({rate*100:.2f}%, ${min_fee})")
        self.rate = rate
        self.min_fee = min_fee
    
    def calculate(self, shares: float, price: float) -> float:
        """Calculate commission as percentage with minimum fee."""
        trade_value = shares * price
        percentage_fee = trade_value * self.rate
        
        # Return whichever is higher: percentage or minimum
        return max(percentage_fee, self.min_fee)

print("✓ MinimumFeeCommission created")

✓ MinimumFeeCommission created


In [10]:
class ProgressiveCommission(Commission):
    """Commission that decreases as trading volume increases.
    
    Simulates monthly volume discounts: commission rate decreases
    with each trade (simplified model).
    """
    
    def __init__(self, base_rate: float = 0.002, min_rate: float = 0.0005, 
                 decay_rate: float = 0.95, name: str = None):
        super().__init__(name=name or f"Progressive({base_rate*100:.2f}%)")
        self.base_rate = base_rate
        self.min_rate = min_rate
        self.decay_rate = decay_rate
        self.current_rate = base_rate
        self.trade_count = 0
    
    def calculate(self, shares: float, price: float) -> float:
        """Calculate commission with progressive discount."""
        trade_value = shares * price
        commission = trade_value * self.current_rate
        
        # Decrease rate for next trade (but not below minimum)
        self.trade_count += 1
        self.current_rate = max(self.current_rate * self.decay_rate, self.min_rate)
        
        return commission

print("✓ ProgressiveCommission created")

✓ ProgressiveCommission created


### Test Custom Commissions

In [11]:
# Test MinimumFeeCommission
min_fee_comm = MinimumFeeCommission(rate=0.001, min_fee=5.0)

print("MinimumFeeCommission Examples:")
print(f"  Trade $1,000:  ${min_fee_comm.calculate(10, 100):.2f} (min fee applies)")
print(f"  Trade $10,000: ${min_fee_comm.calculate(100, 100):.2f} (percentage applies)")
print(f"  Trade $20,000: ${min_fee_comm.calculate(200, 100):.2f} (percentage applies)")

MinimumFeeCommission Examples:
  Trade $1,000:  $5.00 (min fee applies)
  Trade $10,000: $10.00 (percentage applies)
  Trade $20,000: $20.00 (percentage applies)


In [12]:
# Test ProgressiveCommission
prog_comm = ProgressiveCommission(base_rate=0.002, min_rate=0.0005)

print("\nProgressiveCommission Examples (rate decreases with each trade):")
for i in range(5):
    comm = prog_comm.calculate(100, 100)  # $10,000 trade
    print(f"  Trade {i+1}: ${comm:.2f} (rate: {prog_comm.current_rate*100:.4f}%)")


ProgressiveCommission Examples (rate decreases with each trade):
  Trade 1: $20.00 (rate: 0.1900%)
  Trade 2: $19.00 (rate: 0.1805%)
  Trade 3: $18.05 (rate: 0.1715%)
  Trade 4: $17.15 (rate: 0.1629%)
  Trade 5: $16.29 (rate: 0.1548%)


## 5. Commission Impact Analysis

Compare how different commission structures affect the same strategy.

In [13]:
# Create configs with different commission structures
configs = {
    "Zero Commission": BacktestConfig(
        initial_capital=10000.0,
        lookback_period=50,
        commission_type="flat",
        commission_value=0.0,
        risk_free_rate=0.02
    ),
    "Low (0.05%)": BacktestConfig(
        initial_capital=10000.0,
        lookback_period=50,
        commission_type="percentage",
        commission_value=0.0005,
        risk_free_rate=0.02
    ),
    "Medium (0.1%)": BacktestConfig(
        initial_capital=10000.0,
        lookback_period=50,
        commission_type="percentage",
        commission_value=0.001,
        risk_free_rate=0.02
    ),
    "High (0.25%)": BacktestConfig(
        initial_capital=10000.0,
        lookback_period=50,
        commission_type="percentage",
        commission_value=0.0025,
        risk_free_rate=0.02
    ),
    "Flat $5": BacktestConfig(
        initial_capital=10000.0,
        lookback_period=50,
        commission_type="flat",
        commission_value=5.0,
        risk_free_rate=0.02
    ),
}

print("Testing commission impact on MA strategy...")
print(f"Strategy: {strategy.get_name()}\n")

Testing commission impact on MA strategy...
Strategy: MA_10_30



In [14]:
# Run backtests with different commissions
comparison_results = []

for name, config in configs.items():
    backtest = Backtest(data=data, config=config)
    results = backtest.run([strategy])
    
    metrics = results[strategy.get_name()]['metrics']
    
    comparison_results.append({
        'Commission Type': name,
        'Total Return (%)': metrics['total_return'] * 100,
        'CAGR (%)': metrics['cagr'] * 100,
        'Sharpe Ratio': metrics['sharpe_ratio'],
        'Max Drawdown (%)': metrics['max_drawdown'] * 100,
        'Total Trades': metrics['total_trades'],
        'Win Rate (%)': metrics['win_rate'] * 100,
    })

# Create comparison DataFrame
comparison_df = pd.DataFrame(comparison_results)

print("\nCOMMISSION IMPACT ANALYSIS")
print("=" * 100)
print(comparison_df.to_string(index=False))
print("=" * 100)

Running strategies: 100%|██████████| 1/1 [00:00<00:00, 14.16it/s]


COMMISSION IMPACT ANALYSIS
Commission Type  Total Return (%)   CAGR (%)  Sharpe Ratio  Max Drawdown (%)  Total Trades  Win Rate (%)
Zero Commission       1208.685327 305.934535      0.251502        659.976572            31   5333.333333
    Low (0.05%)       1148.988531 291.409920      0.219763        678.710720            31   5333.333333
  Medium (0.1%)       1089.291735 276.827959      0.187991        697.524355            31   5333.333333
   High (0.25%)        910.201347 232.732595      0.092600        754.447266            31   4666.666667
        Flat $5       1053.685327 268.102963      0.169002        708.466746            31   4666.666667


In [15]:
# Calculate commission cost impact
zero_return = comparison_df[comparison_df['Commission Type'] == 'Zero Commission']['Total Return (%)'].values[0]

print("\nCOMMISSION COST IMPACT:")
print("=" * 60)
for _, row in comparison_df.iterrows():
    if row['Commission Type'] != 'Zero Commission':
        impact = zero_return - row['Total Return (%)']
        impact_pct = (impact / zero_return * 100) if zero_return != 0 else 0
        print(f"{row['Commission Type']:.<20} Cost: {impact:>6.2f}% ({impact_pct:>5.1f}% of returns)")
print("=" * 60)


COMMISSION COST IMPACT:
Low (0.05%)......... Cost:  59.70% (  4.9% of returns)
Medium (0.1%)....... Cost: 119.39% (  9.9% of returns)
High (0.25%)........ Cost: 298.48% ( 24.7% of returns)
Flat $5............. Cost: 155.00% ( 12.8% of returns)


## High-Frequency vs Low-Frequency Trading

Commission impact varies dramatically based on trading frequency!

In [16]:
# Compare high-frequency (short MA windows) vs low-frequency (long MA windows)
from simple_backtest.strategy import BuyAndHoldStrategy

strategies_freq = [
    MovingAverageStrategy(short_window=5, long_window=15, shares=10, name="MA_High_Freq"),
    MovingAverageStrategy(short_window=20, long_window=50, shares=10, name="MA_Low_Freq"),
    BuyAndHoldStrategy(shares=50, name="Buy_Hold"),
]

# Test with high commission (0.25%)
config_high_comm = BacktestConfig(
    initial_capital=10000.0,
    lookback_period=60,
    commission_type="percentage",
    commission_value=0.0025,  # 0.25%
    risk_free_rate=0.02
)

backtest = Backtest(data=data, config=config_high_comm)
results_freq = backtest.run(strategies_freq)

print("\nTRADING FREQUENCY vs COMMISSION IMPACT (0.25% commission)")
print("=" * 80)
for strat in strategies_freq:
    metrics = results_freq[strat.get_name()]['metrics']
    print(f"\n{strat.get_name()}:")
    print(f"  Total Trades:  {metrics['total_trades']:>4}")
    print(f"  Total Return:  {metrics['total_return']*100:>7.2f}%")
    print(f"  Sharpe Ratio:  {metrics['sharpe_ratio']:>7.2f}")
print("=" * 80)


TRADING FREQUENCY vs COMMISSION IMPACT (0.25% commission)

MA_High_Freq:
  Total Trades:    61
  Total Return:  1551.76%
  Sharpe Ratio:     0.45

MA_Low_Freq:
  Total Trades:    17
  Total Return:   726.75%
  Sharpe Ratio:     0.00

Buy_Hold:
  Total Trades:     0
  Total Return:     0.00%
  Sharpe Ratio:     0.00


## Summary

In this notebook, we explored commission models:

1. ✅ **Percentage Commission**: Most common, scales with trade size
2. ✅ **Flat Commission**: Fixed fee, benefits larger trades
3. ✅ **Tiered Commission**: Volume discounts for large trades
4. ✅ **Custom Commissions**: Created MinimumFee and Progressive models
5. ✅ **Impact Analysis**: Measured how commissions affect performance

### Key Insights:

**Commission Impact:**
- Higher trading frequency = higher commission costs
- Even "low" commissions (0.1%) can significantly reduce returns
- High-frequency strategies need ultra-low commissions to be profitable
- Buy-and-hold strategies are less affected by commission rates

**Choosing Commission Type:**
- **Percentage**: Good for variable trade sizes
- **Flat**: Better for consistent large trades
- **Tiered**: Best for high-volume trading
- **Custom**: Match your exact broker structure

### Important Considerations:

1. **Don't Forget Commissions**: Always include realistic commission costs
2. **Match Your Broker**: Use the exact commission structure of your broker
3. **Trading Frequency**: High-frequency strategies are extremely sensitive to commissions
4. **Slippage**: Real costs include slippage (not just commission)
5. **Tax Impact**: Consider tax implications (not modeled here)

### Creating Custom Commissions:

To create your own commission model:
```python
from simple_backtest.commission import Commission

class MyCommission(Commission):
    def calculate(self, shares: float, price: float) -> float:
        # Your logic here
        return commission_amount
```

### Real-World Commission Examples:

**US Stock Brokers:**
- Robinhood, Webull: $0
- Interactive Brokers: $0.0035/share (min $0.35)
- TD Ameritrade: $0

**Crypto Exchanges:**
- Coinbase Pro: 0.5% (retail) to 0.04% (high volume)
- Binance: 0.1% (can be reduced with BNB)
- Kraken: 0.16% to 0.26%

**Forex Brokers:**
- Spread-based (2-5 pips)
- Commission: $3-$7 per lot

### Next Steps:

- Add slippage modeling to commissions
- Model market impact for large orders
- Include tax-aware backtesting
- Test strategies across different commission environments